In [70]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv('train.csv')

In [230]:
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from scipy.linalg import lstsq
from numpy.linalg import pinv

In [106]:
X=df.drop(['yield', 'id', 'Row#'], axis=1)
# X=df.drop('yield', axis=1)

In [5]:
y=df['yield']

In [8]:
from sklearn.model_selection import train_test_split

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [60]:
# regressor=LassoCV(cv=5, random_state=0).fit(X_train, y_train)
regressor=LinearRegression().fit(X_train, y_train)

In [61]:
preds=regressor.predict(X_test)

In [62]:
from sklearn.metrics import mean_absolute_error, r2_score

In [63]:
r2_score(y_test, preds)

0.9187345540808263

In [64]:
for coef in regressor.coef_:
    print(coef)

-1.6068705525308618
17.12978511099368
95.29292712886104
13.70827130362426
81.36851808167651
5576.462575585038
5213.768701194413
-673.0527774662925
-14.801445216996306
-23360.79721187751
-13.413460476760584
4.174152392385878
-471.4487122865894
12916.971454761264
-1504.2643868453692
83.76388388149384


In [16]:
from sklearn.preprocessing import PolynomialFeatures

In [48]:
X_poly=PolynomialFeatures(degree=3).fit_transform(X)

In [65]:
mean_absolute_error(y_test, preds)

262.22313984124867

In [231]:
class MultiVariateRegressor():
    def __init__(self, degree):
        self.degree = degree
        self.X = None
        self.Y = None
        self.B = None

    def fit(self, X, Y):
        indices=np.random.shuffle(np.arange(X.shape[0]))
        self.orig_X = X[indices][0]
        self.Y = Y[indices][0]
        self.X = X[indices][0]
        self.X = np.hstack((np.ones((self.X.shape[0], 1)), self.X))
        self.B = np.matmul(pinv(np.matmul(self.X.T, self.X)), np.matmul(self.X.T, self.Y))
        # regres=LinearRegression(fit_intercept=False).fit(self.X, self.Y)
        # for coef in zip(regres.coef_, self.B):
        #     print(coef)

    def generate_powers(self, x):
        powers = [1]
        for i in range(1, self.degree+1):
            powers.append(powers[-1]*x)
        return np.array(powers)
    
    def generate_sorted_vander(self, X, degree):
        num_samples, num_features = X.shape 
        vander_matrix = [np.ones(num_samples)]
        
        for d in range(1, degree+1):
            for indices in np.ndindex(*(num_features,)*d):
                if sum(indices) == d:                
                    vander_matrix.append(np.prod(X ** indices, axis=1))
        return np.column_stack(vander_matrix)
    
    def predict_one(self, x):
        return np.dot(self.generate_powers(x), self.B)

    def predict(self, X):
        return np.dot(np.hstack((np.ones((X.shape[0], 1)), X)), self.B)
        vectorized_predict_one = np.vectorize(self.predict_one)
        return vectorized_predict_one(X)

    def sum_of_squares(self, Y_true, Y_pred):
        return np.sum((Y_true - Y_pred)**2)
        
    def r2_score(self, Y_true, Y_pred):
        # return np.sum((Y_true - Y_pred)**2)
        return 1 - np.sum((Y_true - Y_pred)**2) / np.sum((Y_true - np.mean(Y_true))**2)

    def k_fold_testing(self, k=10):
        width = self.X.shape[0]//k
        average_r2 = 0
        count = 0
        for i in range(0, self.X.shape[0], k):
            count += 1
            X_train = np.concatenate((self.X[:i], self.X[i+width:]))
            Y_train = np.concatenate((self.Y[:i], self.Y[i+width:]))
            X_test = self.X[i:i+width]
            Y_test = self.Y[i:i+width]
            B = np.matmul(np.linalg.inv(np.matmul(X_train.T, X_train)), np.matmul(X_train.T, Y_train))
            predictions = np.matmul(X_test, B)
            # average_r2 += r2_score(Y_test, predictions)
            average_r2 += self.r2_score(Y_test, predictions)
        B = np.matmul(np.linalg.inv(np.matmul(self.X.T, self.X)), np.matmul(self.X.T, self.Y))
        return average_r2/count, np.matmul(self.X, B)
    
    def cross_validation(self, params):
        maximum_param=np.max(params)
        self.max_X=np.vander(self.orig_X, maximum_param+1, increasing=True)
        responses = []
        for param in params:
            self.degree = param
            self.X=self.max_X[:,:param+1]
            response, plot_values = self.k_fold_testing()
            responses.append(response)
            # fig=plt.figure()
            # plt.scatter(self.orig_X, self.Y)
            # indices=np.argsort(self.orig_X)
            # plt.plot(self.orig_X[indices], plot_values[indices], 'red')
            # fig.savefig(f"degree_{param}.png")
        print(responses)

In [233]:
reg=MultiVariateRegressor(3)

In [215]:
reg.fit(X_train.values, y_train.values)

In [216]:
r2_score(y_test, reg.predict(X_test.values))

0.9187345697992927

In [217]:
mean_absolute_error(y_test, reg.predict(X_test.values))

262.2232781809418

In [266]:
def write_output():
    # X_all = np.vstack((X_train, X_test))
    # y_all = np.concatenate((y_train, y_test))
    reg.fit(np.vstack((X_train, X_test)), np.concatenate((y_train, y_test)))
    test_data=pd.read_csv('test.csv')
    X=test_data.drop(['id', 'Row#'], axis=1)
    X=PolynomialFeatures(degree=2).fit_transform(X)
    preds=reg.predict(X)
    test_data['yield']=preds
    test_data=test_data[['id', 'yield']]
    test_data.to_csv('output.csv', index=False)
    

In [267]:
write_output()

In [114]:
from sklearn.tree import DecisionTreeRegressor

In [119]:
kde = LinearRegression().fit(X_train, y_train)

In [120]:
predictions = kde.predict(X_test)

In [121]:
r2_score(y_test, predictions)

0.9187345540808263

In [122]:
mean_absolute_error(y_test, predictions)

262.22313984124867

In [123]:
kde.coef_

array([-1.60687055e+00,  1.71297851e+01,  9.52929271e+01,  1.37082713e+01,
        8.13685181e+01,  5.57646258e+03,  5.21376870e+03, -6.73052777e+02,
       -1.48014452e+01, -2.33607972e+04, -1.34134605e+01,  4.17415239e+00,
       -4.71448712e+02,  1.29169715e+04, -1.50426439e+03,  8.37638839e+01])

In [124]:
regressor.B

array([-4.90231332e+00,  1.40762086e+01, -6.07800130e+01, -5.59329633e+01,
        7.54680576e+01, -7.20406858e+03, -6.14319839e+03, -2.28054561e+03,
        8.95157011e+00,  3.68329640e+04, -7.60610148e+01,  9.46642987e+00,
       -1.02912227e+03,  1.46331629e+04, -6.26779624e+03,  8.13053845e+01])

In [126]:
for coef in zip(kde.coef_, regressor.B):
    print(coef)

(-1.6068705525308618, -4.902313319034874)
(17.12978511099368, 14.076208615791984)
(95.29292712886104, -60.78001298010349)
(13.70827130362426, -55.9329632781446)
(81.36851808167651, 75.46805759519339)
(5576.462575585038, -7204.06858253479)
(5213.768701194413, -6143.1983852386475)
(-673.0527774662925, -2280.545612215996)
(-14.801445216996306, 8.951570108532906)
(-23360.79721187751, 36832.96404647827)
(-13.413460476760584, -76.06101481616497)
(4.174152392385878, 9.466429865453392)
(-471.4487122865894, -1029.12227293849)
(12916.971454761264, 14633.162858784199)
(-1504.2643868453692, -6267.796239495277)
(83.76388388149384, 81.30538450879976)


In [172]:
for coef in zip(kde.coef_, regressor.B):
    print(coef)

(-10.634959249242108, -4.902313319034874)
(3.7138845195848944, 14.076208615791984)
(5.641474684876364, -60.78001298010349)
(2.0100770397065872, -55.9329632781446)
(11.682191180926282, 75.46805759519339)
(50817.1556057072, -7204.06858253479)
(28812.66593439402, -6143.1983852386475)
(-5124.054559419127, -2280.545612215996)
(-97.4832317294927, 8.951570108532906)
(-74360.00371577418, 36832.96404647827)
(-72.081111308513, -76.06101481616497)
(49.375473076937226, 9.466429865453392)
(-78.37846581793987, -1029.12227293849)
(990.6356013384702, 14633.162858784199)
(-57.36946245508608, -6267.796239495277)
(343.35354152807116, 81.30538450879976)


In [138]:
X_train2=PolynomialFeatures(degree=2).fit_transform(X_train)
X_test2=PolynomialFeatures(degree=2).fit_transform(X_test)

In [154]:
kde = LinearRegression().fit(X_train3, y_train)

In [155]:
mean_absolute_error(y_test, kde.predict(X_test3))

262.2231398412343

In [150]:
from sklearn.preprocessing import StandardScaler

In [152]:
ss=StandardScaler().fit(X_train)

In [153]:
X_train3=ss.transform(X_train)
X_test3=ss.transform(X_test)

In [161]:
r2_score(y_test, kde.predict(X_test3))  

0.9187345540808249

In [226]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.values
X_test = X_test.values

# List of models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K-Neighbors Regressor': KNeighborsRegressor()
}

# Evaluate each model
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    results[name] = {'MAE': mae}

# Fit your custom regressor
reg.fit(X_train, y_train.values)
predictions = reg.predict(X_test)
results['Custom Regressor'] = {'MAE': mean_absolute_error(y_test, predictions)}

# Display the results
for model_name, metrics in results.items():
    print(f"{model_name}: MAE = {metrics['MAE']}")


/home/aditya/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.048e+07, tolerance: 2.189e+06
  model = cd_fast.enet_coordinate_descent(


Linear Regression: MAE = 269.7418799099052
Ridge Regression: MAE = 272.18335295439056
Lasso Regression: MAE = 275.59924082556154
ElasticNet Regression: MAE = 351.35732948280616
Random Forest: MAE = 261.89203582335915
Gradient Boosting: MAE = 250.38153643448032
AdaBoost Regressor: MAE = 412.6528484335456
Support Vector Regression: MAE = 1016.4474512745328
Decision Tree Regressor: MAE = 377.53323939666666
K-Neighbors Regressor: MAE = 347.42687868
Custom Regressor: MAE = 269.74190491481926


In [268]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Split your data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train = X_train.values
# X_test = X_test.values
# from sklearn.preprocessing import PolynomialFeatures
# X_train = PolynomialFeatures(degree=2).fit_transform(X_train)
# X_test = PolynomialFeatures(degree=2).fit_transform(X_test)
# from sklearn.preprocessing import StandardScaler
# ss=StandardScaler().fit(X_train)
# # X_train=ss.transform(X_train)
# # X_test=ss.transform(X_test)
X_train=X_train[:, lasso_regressor.coef_ != 0]
X_test=X_test[:, lasso_regressor.coef_ != 0]
# List of models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K-Neighbors Regressor': KNeighborsRegressor()
}

# Evaluate each model
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    results[name] = {'MAE': mae}

# Fit your custom regressor
reg.fit(X_train, y_train.values)
predictions = reg.predict(X_test)
results['Custom Regressor'] = {'MAE': mean_absolute_error(y_test, predictions)}

# Display the results
for model_name, metrics in results.items():
    print(f"{model_name}: MAE = {metrics['MAE']}")


/home/aditya/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.057e+09, tolerance: 2.189e+06
  model = cd_fast.enet_coordinate_descent(


Linear Regression: MAE = 272.1912949141611
Ridge Regression: MAE = 276.34054405664676
Lasso Regression: MAE = 286.82496083118104
ElasticNet Regression: MAE = 291.98583626180795
Random Forest: MAE = 262.41807264920993
Gradient Boosting: MAE = 250.0928636133857
AdaBoost Regressor: MAE = 376.9815276545639
Support Vector Regression: MAE = 1064.88014072126
Decision Tree Regressor: MAE = 369.13922374
K-Neighbors Regressor: MAE = 332.16913450199996
Custom Regressor: MAE = 272.1912949094509


In [235]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Split your data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.values
X_test = X_test.values
from sklearn.preprocessing import PolynomialFeatures
X_train = PolynomialFeatures(degree=2).fit_transform(X_train)
X_test = PolynomialFeatures(degree=2).fit_transform(X_test)
from sklearn.preprocessing import StandardScaler
ss=StandardScaler().fit(X_train)
# X_train=ss.transform(X_train)
# X_test=ss.transform(X_test)

# List of models to evaluate
models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'ElasticNet Regression': ElasticNet(),
    'Random Forest': RandomForestRegressor(),
    'Gradient Boosting': GradientBoostingRegressor(),
    'AdaBoost Regressor': AdaBoostRegressor(),
    'Support Vector Regression': SVR(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'K-Neighbors Regressor': KNeighborsRegressor()
}

# Evaluate each model
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    results[name] = {'MAE': mae}

# Fit your custom regressor
reg.fit(X_train, y_train.values)
predictions = reg.predict(X_test)
results['Custom Regressor'] = {'MAE': mean_absolute_error(y_test, predictions)}

# Display the results
for model_name, metrics in results.items():
    print(f"{model_name}: MAE = {metrics['MAE']}")


Linear Regression: MAE = 270.2113776886595
Ridge Regression: MAE = 269.48967952805043
Lasso Regression: MAE = 270.9443636445984
ElasticNet Regression: MAE = 293.1215223417799
Random Forest: MAE = 262.0827522726089
Gradient Boosting: MAE = 250.20869805098388
AdaBoost Regressor: MAE = 379.67884740463194
Support Vector Regression: MAE = 733.2060281254987
Decision Tree Regressor: MAE = 374.4861873733333
K-Neighbors Regressor: MAE = 359.207866724
Custom Regressor: MAE = 269.63626413551


In [234]:
reg.fit(X_train, y_train.values)
predictions = reg.predict(X_test)
results['Custom Regressor'] = {'MAE': mean_absolute_error(y_test, predictions)}


In [236]:
lasso_regressor=LassoCV(cv=5, random_state=0).fit(X_train, y_train)

In [248]:
for coef_ in zip(lasso_regressor.coef_[np.where(lasso_regressor.coef_ != 0)], reg.B):
    print(coef_)

(-54.14244296508789, 6024.759223668327)
(-0.21613085862132517, -291.1603576114285)
(1080.5115281413478, 87.06694569255342)
(107.77490325611852, 1344.069354611478)
(-28.084439783639002, -44.630208914750256)
(-0.2552188511842227, -19.288134651962537)
(5.032552408092508, -1.480891664019964)
(-2.537911648139373, 5.8726740401252755)
(7.819160894141544, -17.50426606231258)
(-8.647315189639444, 14.30190516337143)
(3.3849725539192588, -15.242690700908497)
(-16.472690067045765, 7.464691819624022)
(30.015904858741685, -30.379559525024888)
(16.145607616105753, 38.04968245529744)
(-20.58284669928821, 235.52401351122535)
(-70.84311245741726, 624.158798167482)
(61.782089013794206, -1101.1216925410554)
(-107.09956924342916, 222.95162866868486)
(225.48153851641524, -112.29805905175704)


In [240]:
X_train2=X_train[:,lasso_regressor.coef_ != 0]
X_test2=X_test[:,lasso_regressor.coef_ != 0]

In [244]:
print(X_train2.shape, y_train.shape)

(12000, 19) (12000,)


In [246]:
reg.fit(X_train2, y_train.values)

In [247]:
mean_absolute_error(y_test, reg.predict(X_test2))

272.19129491395967